In [1]:
import numpy as np
import pandas as pd
np.random.seed(2019)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

## 进阶Pandas高级用用

### 复习数据规整流程 以及 Numpy 基本操作 和 进阶用法

## 2.1 分类数据

In [2]:
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values) # unique 用法 

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values) # value_counts用法

apple     6
orange    2
dtype: int64

In [6]:
values = pd.Series([0, 1, 0, 0] * 2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [7]:
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

In [8]:
# 使用take方法存储原始的字符串Series
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### pandas的分类类型

In [10]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
N

8

In [11]:
df = pd.DataFrame({'fruit': fruits,\
                   'basket_id': np.arange(N),\
                   'count': np.random.randint(3, 15, size=N),\
                   'weight': np.random.uniform(0, 4, size=N)},\
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.719876
1,1,orange,5,3.553092
2,2,apple,8,2.690503
3,3,apple,11,0.085292
4,4,apple,9,1.814268
5,5,orange,11,0.196478
6,6,apple,13,3.446388
7,7,apple,3,0.487360


In [14]:
# df['fruit']是一个Python字符串对象的数组
print(type(df['fruit']))
df['fruit']

<class 'pandas.core.series.Series'>


0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [15]:
fruit_cat = df['fruit'].astype('category') # 将其转变为分类
# fruit_cat的值不是NumPy数组，而是一个pandas.Categorical实例
fruit_cat 

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [16]:
# fruit_cat的值不是NumPy数组，而是一个pandas.Categorical实例
c = fruit_cat.values
type(c)

pandas.core.categorical.Categorical

In [17]:
# 分类对象有categories和codes属性
c.categories # categories属性

Index(['apple', 'orange'], dtype='object')

In [18]:
c.codes # codes属性 类似于 one-hot编码 但有不同

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [19]:
# 将DataFrame的列通过分配转换结果，转换为分类
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [20]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

In [21]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories) # 可以使用from_codes构造器
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [22]:
ordered_cat = pd.Categorical.from_codes(codes, categories,ordered=True) # 指定分类顺序
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [13]:
my_cats_2.as_ordered() # 无需的分类数据
# 分类数据不需要字符串 分类数组可以包括任意不可变类型

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### 用分类进行计算

In [23]:
np.random.seed(2019)
draws = np.random.randn(1000)
draws[:5]

array([-0.21767896,  0.82145535,  1.48127781,  1.33186404, -0.36186537])

In [24]:
bins = pd.qcut(draws, 4) # 使用pandas.qcut面元函数
bins

[(-0.69, 0.0122], (0.702, 2.928], (0.702, 2.928], (0.702, 2.928], (-0.69, 0.0122], ..., (-3.206, -0.69], (-3.206, -0.69], (0.0122, 0.702], (0.702, 2.928], (-0.69, 0.0122]]
Length: 1000
Categories (4, interval[float64]): [(-3.206, -0.69] < (-0.69, 0.0122] < (0.0122, 0.702] < (0.702, 2.928]]

In [25]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
# 使用labels参数qcut 生成汇总
bins
bins.codes[:10]

array([1, 3, 3, 3, 1, 2, 2, 2, 1, 3], dtype=int8)

In [27]:
# 加上标签的面元分类不包含数据面元边界的信息
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

,quartile,count,min,max
0,Q1,250,-3.205040,-0.692267
1,Q2,250,-0.688914,0.009817
2,Q3,250,0.014683,0.701619
3,Q4,250,0.703177,2.928164


In [28]:
# 分位数列保存了原始的面元分类信息  包含排序
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### 用分类提高性能

In [29]:
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [30]:
# 将标签转换为分类
categories = labels.astype('category')

In [31]:
# 标签使用的内存远比分类多
labels.memory_usage()

80000080

In [32]:
categories.memory_usage() # 标签使用的内存远比分类多

10000272

In [33]:
# GroupBy使用分类操作明显更快，是因为底层的算法使用整数编码数组，而不是字符串数组
%time _ = labels.astype('category')

Wall time: 974 ms


### 分类方法

In [34]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [35]:
cat_s.cat.codes # cat属性提供了分类方法的入口

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [36]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [37]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories) # 使用set_categories方法改变数据值
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

In [38]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [39]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In [40]:
# 使用remove_unused_categories方法删除没看到的分类
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [41]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

### 为建模创建虚拟变量（one-hot编码）

In [42]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [43]:
# pandas.get_dummies函数可以转换这个分类数据为包含虚拟变量的DataFrame
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 2.2 GroupBy高级应用

### 分组转换和“解封”GroupBy

In [44]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,\
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [45]:
g = df.groupby('key').value # 按键进行分组
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [46]:
g.transform(lambda x: x.mean()) # 传递函数lambda x: x.mean()进行转换

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [47]:
# 对于内置的聚合函数，我们可以传递一个字符串假名作为GroupBy的agg方法
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [48]:
# 与apply类似，transform的函数会返回Series，但是结果必须与输入大小相同
g.transform(lambda x: x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [49]:
# 计算每个分组的降序排名
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [50]:
# 由聚合构造的的分组转换函数
def normalize(x):
    return (x - x.mean()) / x.std()

In [51]:
# 用transform或apply可以获得等价的结果
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [52]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [38]:
# 内置的聚合函数，比如mean或sum，通常比apply函数快，也比transform快
g.transform('mean')

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [53]:
# 利用解封（unwrapped）分组操作
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized # 解封分组操作可能包括多个分组聚合

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

### 分组的时间重采样

In [ ]:
# 对于时间序列数据，resample方法从语义上是一个基于内在时间的分组操作
N = 15
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)
df = pd.DataFrame({'time': times,\
                   'value': np.arange(N)})
df

In [40]:
# 用time作为索引，然后重采样
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [41]:
# 假设DataFrame包含多个时间序列，用一个额外的分组键的列进行标记
df2 = pd.DataFrame({'time': times.repeat(3),\
                    'key': np.tile(['a', 'b', 'c'], N),\
                    'value': np.arange(N * 3.)})
df2[:7]

,key,time,value
0,a,2017-05-20 00:00:00,0.0
1,b,2017-05-20 00:00:00,1.0
2,c,2017-05-20 00:00:00,2.0
3,a,2017-05-20 00:01:00,3.0
4,b,2017-05-20 00:01:00,4.0
5,c,2017-05-20 00:01:00,5.0
6,a,2017-05-20 00:02:00,6.0


In [42]:
# 要对每个key值进行相同的重采样，可以引入pandas.TimeGrouper对象
time_key = pd.TimeGrouper('5min')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


In [43]:
# 设定时间索引，用key和time_key分组，然后聚合
resampled = (df2.set_index('time')
             .groupby(['key', time_key])
             .sum())
resampled

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0


In [ ]:
resampled.reset_index()

## 2.3 链式编程技术

> **df = load_data()**

> **df2 = df[df['col2'] < 0]**

> **df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()**

> **result = df2.groupby('key').col1_demeaned.std()**

### DataFrame.assign方法是一个df[k] = v形式的函数式的列分配方法

### 它不是就地修改对象，而是返回新的修改过的DataFrame

> **df2 = df.copy()**

> **df2['k'] = v**

> **等价于**

> **df2 = df.assign(k=v)**


### 就地分配可能会比assign快，但是assign可以方便地进行链式编程

> **result = (df2.assign(col1_demeaned=df2.col1 - df2.col2.mean())**
           **.groupby('key')**
          **.col1_demeaned.std())**

### assign和许多其它pandas函数可以接收类似函数的参数，即可调用对象（callable）

> **df = load_data()**

> **df2 = df[df['col2'] < 0]**

### 重写为：

> **df = (load_data()**
       **[lambda x: x['col2'] < 0])**
       
> **load_data的结果没有赋值给某个变量，因此传递到[ ]的函数在这一步被绑定到了对象**

> **result = (load_data()**

>           [lambda x: x.col2 < 0]

>          .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())

>           .groupby('key')

>           .col1_demeaned.std())

### 管道方法

> **a = f(df, arg1=v1)**

> **b = g(a, v2, arg3=v3)**

> **c = h(b, arg4=v4)**

result = (df.pipe(f, arg1=v1)
          .pipe(g, v2, arg3=v3)
          .pipe(h, arg4=v4))
f(df)和df.pipe(f)是等价的，但是pipe使得链式声明更容易

提炼操作为可复用的函数
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')

转换多列，并修改分组的键
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result

In [ ]:
等价于
result = (df[df.col1 < 0]
          .pipe(group_demean, ['key1', 'key2'], ['col1']))

## 2.4 Conclusion

> ### 深入学习Pandas进阶知识 查阅官方文档，结合机器学习或数据挖掘实战项目巩固已学知识